In [1]:
import tensorflow as tf
from tensorflow import keras

2023-10-22 09:40:40.308898: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-22 09:40:41.373739: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-22 09:40:43.913871: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# Suggested structure of the custom loss function
tlist = ... # Which will be used in the custom loss

@keras.saving.register_keras_serializable()
def custom_loss(y, y_pred):
    """
    A custom loss function computing error in energy conservation.
    """

    # is there a better way?
    t = tf.convert_to_tensor(np.linspace(X_train[0,0,0], X_train[0,-1,0], X_train[0,:,0].size).reshape(-1,1), dtype = tf.float32)
    
    predicted_positions = get_trajectories(y_pred) # Obtained from y_pred
    predicted_velocities = tf_compute_velocities(t, predicted_positions) # Obtained from predicted_positions, tlist

    initial_potential_energy = tf_compute_potential_energy(*get_trajectories(y)) # Obtained from positions (y)

    ke_predicted_trajectory = tf_compute_kinetic_energy(predicted_velocities) # Obtained from predicted_velocities
    pe_predicted_trajectory = tf_compute_potential_energy(*predicted_positions) # Obtained from predicted_positions

    energy_error = (ke_predicted_trajectory + pe_predicted_trajectory -
                     initial_potential_energy)

    energy_loss = tf.reduce_mean(tf.abs(energy_error))
    # The relative weight ofthe two terms in the custom loss might be tuned.
    return tf.keras.losses.MeanAbsoluteError()(y, y_pred) + 0.001*tf.clip_by_value(energy_error, -1e6, 1e6)

In [7]:
# Insert the name of the model you want to load
model_names = ["1910_2119_final", "Eloss_1910_0850_final"]
for model_name in model_names:
    model = keras.models.load_model(f"./models/NN_project2_{model_name}.keras")
    model.save(f"./models/NN_project2_{model_name}.h5")

/home/mhuebl/miniconda3/envs/tif285-env/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
